In [1]:
from estimator import NeuralPRFEstimator
import pyterrier as pt
import os
os.environ["JAVA_HOME"] = "/local/trmaster/opt/jdk-11.0.6/"
if not pt.started():
    pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"], mem=8000)
pt.cache.CACHE_DIR = "/users/tr.%s/pyterrier_transformer_cache" % (os.environ["USER"])

Using TensorFlow backend.


In [2]:
topics = pt.get_dataset("trec-robust-2004").get_topics()

In [3]:
topics.head()

,qid,query
0,301,international organized crime
1,302,poliomyelitis and post polio
2,303,hubble telescope achievements
3,304,endangered species mammals
4,305,most dangerous vehicles


In [4]:
topicsTrain=topics[topics.qid.astype(int) % 2 ==0]
topicsTest=topics[topics.qid.astype(int) % 2 !=0]
topicsVal=topicsTrain[100:]
topicsTrain=topicsTrain[:100]

In [5]:
topicsTrain.shape, topicsTest.shape, topicsVal.shape

((100, 2), (125, 2), (25, 2))

In [6]:
qrels = pt.get_dataset("trec-robust-2004").get_qrels()

In [7]:
# pt.ApplicationSetup.setProperty("termpipelines", "")
index_location = "/users/tr.craigm/projects/tr5/Terrier/var/index_disk45/data.properties"

eval_metrics = ["map", "num_rel_ret", "ndcg", "ndcg_cut_10", "recip_rank"]

In [8]:
index = pt.IndexFactory.of(index_location)
print(index.getCollectionStatistics().toString())

Number of documents: 528155
Number of terms: 520423
Number of fields: 0
Number of tokens: 143669447
Field names: []
Positions:   false



In [9]:
# from pyterrier.rewrite import RM3, AxiomaticQE

# firstpass_dph = pt.BatchRetrieve(index, wmodel="DPH", 
#                                  properties={"termpipelines": ""},
#                                  verbose=True, metadata=["docno"])
firstpass_dph = pt.BatchRetrieve(index, wmodel="DPH", num_results=100)

In [10]:
# res_train=(firstpass_dph % 100).transform(topicsTrain)
# res_test= (firstpass_dph % 100).transform(topicsTest)
res_train = firstpass_dph.transform(topicsTrain)
res_test= firstpass_dph.transform(topicsTest)
res_val = firstpass_dph.transform(topicsVal)

In [11]:
res_train.shape, res_test.shape, res_val.shape

((9757, 6), (12448, 6), (2500, 6))

In [12]:
eval_metrics = ["map", "num_rel_ret", "ndcg", "ndcg_cut_10", "recip_rank", "ndcg_cut_20"]
pt.Utils.evaluate(res_test, qrels, metrics=eval_metrics)

{'map': 0.2112598609762024,
 'num_rel_ret': 2253.0,
 'ndcg': 0.39133721102085967,
 'ndcg_cut_10': 0.4533752230782252,
 'recip_rank': 0.6737622155622158,
 'ndcg_cut_20': 0.4212950156332943}

In [13]:
import os
nrf = NeuralPRFEstimator(index,'data_robust_100_2')

In [ ]:
nrf.fit(res_train,qrels,res_val, qrels)

parse_idf_for_document: 100%|██████████| 125/125 [00:52<00:00,  2.40q/s]
2020-12-18 00:13:34,298 : INFO : loading projection weights from /users/craigm/public_html/GoogleNews-vectors-negative300.bin.gz
2020-12-18 00:15:52,009 : INFO : loaded (3000000, 300) matrix from /users/craigm/public_html/GoogleNews-vectors-negative300.bin.gz
hist_per_query 302: 100%|██████████| 100/100 [00:00<00:00, 6534.81d/s]

hist_per_query 304: 100%|██████████| 100/100 [00:00<00:00, 7050.32d/s]

hist_per_query 306: 100%|██████████| 100/100 [00:00<00:00, 6979.57d/s]

hist_per_query 308: 100%|██████████| 100/100 [00:00<00:00, 7226.45d/s]

hist_per_query 310: 100%|██████████| 100/100 [00:00<00:00, 7081.74d/s]

hist_per_query 312: 100%|██████████| 21/21 [00:00<00:00, 6049.89d/s]

hist_per_query 316: 100%|██████████| 30/30 [00:00<00:00, 6179.00d/s]

hist_per_query 318: 100%|██████████| 100/100 [00:00<00:00, 6977.83d/s]

hist_per_query 320: 100%|██████████| 100/100 [00:00<00:00, 6855.68d/s]

hist_per_query 322: 100

hist_per_query 642: 100%|██████████| 100/100 [00:00<00:00, 5771.40d/s]

hist_per_query 644: 100%|██████████| 100/100 [00:00<00:00, 5631.15d/s]

hist_per_query 646: 100%|██████████| 100/100 [00:00<00:00, 5518.02d/s]

hist_per_query 648: 100%|██████████| 100/100 [00:00<00:00, 5679.26d/s]

hist_per_query 650: 100%|██████████| 100/100 [00:00<00:00, 5792.68d/s]

hist_per_query 654: 100%|██████████| 100/100 [00:00<00:00, 5879.15d/s]

hist_per_query 656: 100%|██████████| 100/100 [00:00<00:00, 5973.94d/s]

hist_per_query 658: 100%|██████████| 100/100 [00:00<00:00, 5770.36d/s]

hist_per_query 660: 100%|██████████| 100/100 [00:00<00:00, 5621.34d/s]

hist_per_query 664: 100%|██████████| 100/100 [00:00<00:00, 5944.31d/s]

hist_per_query 666: 100%|██████████| 100/100 [00:00<00:00, 5800.21d/s]

hist_per_query 668: 100%|██████████| 100/100 [00:00<00:00, 5904.23d/s]

hist_per_query 670: 100%|██████████| 100/100 [00:00<00:00, 5997.17d/s]

hist_per_query 672: 100%|██████████| 100/100 [00:00<00:00, 6136.

Finish all!



2020-12-18 00:15:57,642 : WARNING : From /users/tr.saraw/anaconda/envs/pyterrier/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



2020-12-18 00:15:57,649 : WARNING : From /users/tr.saraw/anaconda/envs/pyterrier/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



...

[Iter 288]	Loss: [0.5793988479859039]
[Valid]		MAP	P20	NDCG20
		0.2377	0.41	0.4204
iteration: 289 2020-12-18 14:01:54.517365
[Iter 289]	Loss: [0.5794401892567297]
[Valid]		MAP	P20	NDCG20
		0.2382	0.41	0.4209
iteration: 290 2020-12-18 14:04:48.052221
[Iter 290]	Loss: [0.5793452323713434]
[Valid]		MAP	P20	NDCG20
		0.2352	0.41	0.4146
iteration: 291 2020-12-18 14:07:42.095171
[Iter 291]	Loss: [0.5794613018532511]
[Valid]		MAP	P20	NDCG20
		0.2356	0.41	0.4150
iteration: 292 2020-12-18 14:10:35.874583
[Iter 292]	Loss: [0.5793716998660534]
[Valid]		MAP	P20	NDCG20
		0.2353	0.41	0.4140
iteration: 293 2020-12-18 14:13:30.138457
[Iter 293]	Loss: [0.5792463678583888]
[Valid]		MAP	P20	NDCG20
		0.2352	0.40	0.4131
iteration: 294 2020-12-18 14:16:21.696328
[Iter 294]	Loss: [0.5793366919740028]
[Valid]		MAP	P20	NDCG20
		0.2354	0.40	0.4128
iteration: 295 2020-12-18 14:19:12.149350
[Iter 295]	Loss: [0.5793681861001616]
[Valid]		MAP	P20	NDCG20
		0.2352	0.40	0.4127
iteration: 296 2020-12-18 14:22:05.382407


2020-12-18 15:07:45,826 : INFO : Generate totally 47950 pairs


[Iter 312]	Loss: [0.5779501433586011]
[Valid]		MAP	P20	NDCG20
		0.2348	0.41	0.4126
iteration: 313 2020-12-18 15:10:39.847848
[Iter 313]	Loss: [0.5777653814421608]
[Valid]		MAP	P20	NDCG20
		0.2337	0.41	0.4118
iteration: 314 2020-12-18 15:13:33.683632
[Iter 314]	Loss: [0.5777296158186066]
[Valid]		MAP	P20	NDCG20
		0.2337	0.41	0.4122
iteration: 315 2020-12-18 15:16:27.318798
[Iter 315]	Loss: [0.57776849493664]
[Valid]		MAP	P20	NDCG20
		0.2337	0.41	0.4121
iteration: 316 2020-12-18 15:19:20.415847
[Iter 316]	Loss: [0.5779635342190647]
[Valid]		MAP	P20	NDCG20
		0.2337	0.41	0.4121
iteration: 317 2020-12-18 15:22:12.106849
[Iter 317]	Loss: [0.5777488089662924]
[Valid]		MAP	P20	NDCG20
		0.2337	0.41	0.4121
iteration: 318 2020-12-18 15:25:06.748956
[Iter 318]	Loss: [0.577746345483705]
[Valid]		MAP	P20	NDCG20
		0.2341	0.41	0.4123
iteration: 319 2020-12-18 15:27:59.543925
[Iter 319]	Loss: [0.5776984037565778]
[Valid]		MAP	P20	NDCG20
		0.2342	0.41	0.4124
iteration: 320 2020-12-18 15:30:52.240624
[It

2020-12-18 16:16:39,840 : INFO : Generate totally 47950 pairs


[Iter 336]	Loss: [0.5795893626282919]
[Valid]		MAP	P20	NDCG20
		0.2353	0.41	0.4151
iteration: 337 2020-12-18 16:19:32.658978
[Iter 337]	Loss: [0.5795998414038518]
[Valid]		MAP	P20	NDCG20
		0.2352	0.41	0.4147
iteration: 338 2020-12-18 16:22:26.234504
[Iter 338]	Loss: [0.5795646496148185]
[Valid]		MAP	P20	NDCG20
		0.2349	0.41	0.4146
iteration: 339 2020-12-18 16:25:20.984561
[Iter 339]	Loss: [0.5795217215194615]
[Valid]		MAP	P20	NDCG20
		0.2350	0.41	0.4139
iteration: 340 2020-12-18 16:28:08.486121
[Iter 340]	Loss: [0.5794986277396128]
[Valid]		MAP	P20	NDCG20
		0.2350	0.41	0.4145
iteration: 341 2020-12-18 16:30:59.565147
[Iter 341]	Loss: [0.5794505435069269]
[Valid]		MAP	P20	NDCG20
		0.2348	0.41	0.4139
iteration: 342 2020-12-18 16:33:53.902710
[Iter 342]	Loss: [0.5793195104038497]
[Valid]		MAP	P20	NDCG20
		0.2350	0.41	0.4157
iteration: 343 2020-12-18 16:36:46.262846
[Iter 343]	Loss: [0.5795501289179268]
[Valid]		MAP	P20	NDCG20
		0.2351	0.41	0.4157
iteration: 344 2020-12-18 16:39:39.634058


In [ ]:
# import pandas as pd
# res = pd.concat([res_train, res_val])

In [ ]:
# res[res['docid']==395066]

In [ ]:
# topk = nrf.top_k(res)

In [ ]:
# res.head()

In [ ]:
# topk.keys[]

In [ ]:
# relevance_dict=nrf.create_relevance(res_train,qrels)
# topk_term_corpus = nrf.top_k(res_train,20)
# top_idf_dict = nrf.parse_idf_for_document(relevance_dict,topk_term_corpus, rerank_topk=60, doc_topk_term=20)
        


In [ ]:
# nrf.preprocess(res_train, qrels, topicsTrain, 20)

In [ ]:
# relevance_dict=nrf.create_relevance(res_train,qrels)
# topk_term_corpus = nrf.top_k(res_train,20)
# top_idf_dict = nrf.parse_idf_for_document(relevance_dict,topk_term_corpus, rerank_topk=60, doc_topk_term=20)


In [ ]:
# nrf.preprocess(res_test, qrels, topicsTest, 20)

In [ ]:
res_t = nrf.transform(res_test, topicsTest)

In [ ]:
res_t

In [ ]:
eval_metrics = ["map", "num_rel_ret", "ndcg", "ndcg_cut_10", "recip_rank", "ndcg_cut_20"]
eval_result = pt.Utils.evaluate(res_t,qrels, metrics=eval_metrics)
eval_result

In [ ]:
dataset="trec-deep-learning-docs"
trecdl = pt.get_dataset(dataset)

qrelsTest = trecdl.get_qrels("test")
#this one-liner removes topics from the test set that do not have relevant documents
topicsTest = trecdl.get_topics("test").merge(qrelsTest[qrelsTest["label"] > 0][["qid"]].drop_duplicates())

#we are using an unstemmed index (no stopwords removed)
pt.ApplicationSetup.setProperty("termpipelines", "")
index_location = "/local/terrier/Indices/msMarco/formattedIndexingNoStem/data.properties"

In [ ]:
index = pt.IndexFactory.of(index_location)
print(index.getCollectionStatistics().toString())

In [ ]:
firstpass_dph2 = pt.BatchRetrieve(index, wmodel="DPH", 
                                 properties={"termpipelines": ""},
                                 verbose=True, metadata=["docno"])

In [ ]:
res_test= (firstpass_dph2 % 100).transform(topicsTest)

In [ ]:
# nrf.index = index
# nrf.preprocess(res_test, qrelsTest, topicsTest, 20)

In [ ]:
# nrf.transform(res_test, qrelsTest, topicsTest)

In [ ]:
eval_metrics = ["map", "num_rel_ret", "ndcg", "ndcg_cut_10", "recip_rank",'ndcg_cut_20']
eval_results = pt.Utils.evaluate(res_test,qrelsTest, metrics=eval_metrics)
eval_results